<a href="https://colab.research.google.com/github/Ecaballero333/Obligatorio_ML/blob/develop/VallarinoIgnacio_CaballeroEduardo_Obligatorio_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obligatorio



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files 


uploaded = files.upload()

Saving nuevo_dataset.csv to nuevo_dataset.csv


In [3]:
df=pd.read_csv("nuevo_dataset.csv", sep = ",")

## Funciones


In [6]:
def evaluateAccuracy(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    print(f"ACCURACY TRAIN: {accuracy_score(y_train, y_train_pred):.4f}")
    print(f"ACCURACY TEST: {accuracy_score(y_test, y_test_pred):.4f}")
    print("\n")
    

## Análisis exploratorio

In [5]:
df.head(100)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,79510,Female,22,1,19.0,1,< 1 Year,No,54559.0,152.0,87,0
1,186555,Male,57,1,18.0,0,1-2 Year,Yes,33439.0,124.0,63,0
2,158037,Female,64,1,28.0,0,1-2 Year,Yes,39327.0,122.0,45,0
3,343706,Male,52,1,41.0,1,1-2 Year,No,26536.0,124.0,161,0
4,137903,Female,21,1,7.0,0,< 1 Year,No,28416.0,152.0,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,196823,Male,51,1,28.0,0,1-2 Year,Yes,54154.0,26.0,272,0
96,293262,Male,35,1,46.0,0,1-2 Year,No,22241.0,154.0,267,0
97,260666,Male,24,1,14.0,1,< 1 Year,No,16288.0,152.0,236,0
98,14316,Female,23,1,13.0,1,< 1 Year,No,28760.0,152.0,166,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93891 entries, 0 to 93890
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    93891 non-null  int64  
 1   Gender                93891 non-null  object 
 2   Age                   93891 non-null  int64  
 3   Driving_License       93891 non-null  int64  
 4   Region_Code           93891 non-null  float64
 5   Previously_Insured    93891 non-null  int64  
 6   Vehicle_Age           93891 non-null  object 
 7   Vehicle_Damage        93891 non-null  object 
 8   Annual_Premium        93891 non-null  float64
 9   Policy_Sales_Channel  93891 non-null  float64
 10  Vintage               93891 non-null  int64  
 11  Response              93891 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 8.6+ MB


### Conversión de variables categóricas

Variables que están definidas como número pero son categoricas:
* Driving_License: valores posibles -> 0 y 1
* Region_Code: Es categorica porque es un código
* Previously_Insured: valores posibles -> 0 y 1
* Policy_Sales_Channel: Es categorica porque es un código
* Response: Es el target. Valores posibles -> 0 y 1


In [8]:
# Conversión variable categórica
df['Driving_License'] = df['Driving_License'].astype('str')
df['Region_Code'] = df['Region_Code'].astype('str')
df['Previously_Insured'] = df['Previously_Insured'].astype('str')
df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype('str')
df['Response'] = df['Response'].astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93891 entries, 0 to 93890
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    93891 non-null  int64  
 1   Gender                93891 non-null  object 
 2   Age                   93891 non-null  int64  
 3   Driving_License       93891 non-null  object 
 4   Region_Code           93891 non-null  object 
 5   Previously_Insured    93891 non-null  object 
 6   Vehicle_Age           93891 non-null  object 
 7   Vehicle_Damage        93891 non-null  object 
 8   Annual_Premium        93891 non-null  float64
 9   Policy_Sales_Channel  93891 non-null  object 
 10  Vintage               93891 non-null  int64  
 11  Response              93891 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 8.6+ MB


### Análisis de variables numéricas

In [9]:
df.describe(include = [np.number])

,id,Age,Annual_Premium,Vintage
count,93891.000000,93891.000000,93891.000000,93891.000000
mean,190515.346806,40.806520,30983.086547,154.679415
std,109947.403429,14.368874,17687.544914,83.567929
min,1.000000,20.000000,2630.000000,10.000000
25%,95254.500000,27.000000,24506.000000,82.000000
50%,190640.000000,41.000000,32232.000000,155.000000
75%,285907.500000,50.000000,40299.000000,227.000000
max,381098.000000,85.000000,540165.000000,299.000000


sadasdda